In [1]:
import pandas as pd
from constants import DATA_PATH, EOS_FILE, SENTINEL_FILE

sentinel = pd.read_csv(DATA_PATH / SENTINEL_FILE)
eos = pd.read_csv(DATA_PATH / EOS_FILE)

In [2]:
sentinel = sentinel[sentinel['SM1 (%)'] != 50]
eos = eos[eos['SM1 (%)'] != 50]

In [3]:
from constants import X_cols_eos, X_cols_sentinel, y_col

X_sentinel = sentinel[X_cols_sentinel].values
X_eos = eos[X_cols_eos].values

y_sentinel = sentinel[y_col].values
y_eos = eos[y_col].values

In [4]:
import tensorflow as tf

2025-11-17 00:33:19.876596: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
models = {
#     "16, 1": tf.keras.Sequential([
#     # Input layer
#     tf.keras.Input(shape=(2, )),
#     tf.keras.layers.Dense(16, activation='relu'),
#     tf.keras.layers.Dense(1)
# ]),
#     "8, 1": tf.keras.Sequential([
#     # Input layer
#     tf.keras.Input(shape=(2, )),
#     tf.keras.layers.Dense(8, activation='relu'),
#     tf.keras.layers.Dense(1)
# ]),
#     "2, 1": tf.keras.Sequential([
#     # Input layer
#     tf.keras.Input(shape=(2, )),
#     tf.keras.layers.Dense(2, activation='relu'),
#     tf.keras.layers.Dense(1)
# ]),
#     "4, 1": tf.keras.Sequential([
#     # Input layer
#     tf.keras.Input(shape=(2, )),
#     tf.keras.layers.Dense(4, activation='relu'),
#     tf.keras.layers.Dense(1)
# ]),
    "16, Dropout, 8, Dropout": tf.keras.Sequential([
    # Input layer
    tf.keras.Input(shape=(2, )),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.09),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dropout(0.09),
    tf.keras.layers.Dense(1)
]),
    "16, Dropout": tf.keras.Sequential([
    # Input layer
    tf.keras.Input(shape=(2, )),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1)
])
}

In [6]:
from model_experiments import PredictionIntervalEstimation

tf.keras.backend.clear_session()

eos_results = {}

for param_string, model in models.items():
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

    exp = PredictionIntervalEstimation(X_eos, y_eos, satellite="EOS-04")
    results = exp.run_experiment(model, model_param_string=param_string, optimizer=optimizer, epochs=1000)
    eos_results[param_string] = results

Epochs:   4%|▍         | 38/1000 [00:05<02:12,  7.27epoch/s, loss=0.4376, val_loss=0.4343]

1/7 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
16, Dropout, 8, Dropout: {
    "val": {
        "PICP": 0.9705882352941176,
        "MPIW": 40.6114387512207
    },
    "test": {
        "PICP": 0.9707317073170731,
        "MPIW": 40.932010650634766
    }
}


Epochs:   2%|▏         | 24/1000 [00:03<02:14,  7.26epoch/s, loss=0.4375, val_loss=0.4342]

1/7 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
16, Dropout: {
    "val": {
        "PICP": 0.9705882352941176,
        "MPIW": 40.63493728637695
    },
    "test": {
        "PICP": 0.9707317073170731,
        "MPIW": 40.94243621826172
    }
}


In [7]:
from constants import OUTPUT_PATH
import json

with open(OUTPUT_PATH / "pi_estimation" / "EOS-04_metrics.json", "w") as f:
    json.dump(eos_results, f, indent=4)

In [7]:
from model_experiments import PredictionIntervalEstimation
sentinel_results = {}

for param_string, model in models.items():
    tf.keras.backend.clear_session()
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

    exp = PredictionIntervalEstimation(X_sentinel, y_sentinel, satellite="Sentinel-1")
    results = exp.run_experiment(model, model_param_string=param_string, optimizer=optimizer, epochs=1000)
    sentinel_results[param_string] = results

Epochs:   2%|▏         | 17/1000 [00:02<02:26,  6.71epoch/s, loss=0.4626, val_loss=0.4769]

1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/stepWARNING:tensorflow:5 out of the last 25 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x7f0c47e46a20> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
16, Dropout, 8, Dropout: {
    "val": {
        "PICP": 0.9473684210526315,
        "MPI

Epochs:   2%|▏         | 24/1000 [00:02<01:56,  8.37epoch/s, loss=0.4664, val_loss=0.4768]

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 


1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/stepWARNING:tensorflow:5 out of the last 21 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x7f0c45800c20> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
16, Dropout: {
    "val": {
        "PICP": 0.9473684210526315,
        "MPIW": 40.24116134643555
    },
    "test": {
       

In [9]:
from constants import OUTPUT_PATH
import json

with open(OUTPUT_PATH / "pi_estimation" / "Sentinel-1_metrics.json", "w") as f:
    json.dump(sentinel_results, f, indent=4)